In [1]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import os
import sys

# ==========================================
# 1. 設定參數
# ==========================================
JSON_FILE = "/kaggle/input/knowledge/mmlu.json"         # 您的本地檔案
MODEL_NAME = "mistralai/Mistral-7B-v0.1"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 訓練參數
BATCH_SIZE = 16
EPOCHS = 50
LEARNING_RATE = 0.001

# ==========================================
# 2. 定義 MLP 探測器 (Improved Probe)
# ==========================================
class ImprovedProbe(nn.Module):
    def __init__(self, input_dim, hidden_dim=512):
        super(ImprovedProbe, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# ==========================================
# 3. 輔助函數
# ==========================================
def format_json_prompt(item):
    """
    適配 choices 為 dictionary 的格式
    """
    prompt = f"Question: {item['question']}\n"
    for key in ['A', 'B', 'C', 'D']:
        choice_text = item['choices'].get(key, "")
        prompt += f"{key}. {choice_text}\n"
    prompt += "Answer:"
    return prompt

def parse_model_output(output_text):
    """
    嚴格解析模型輸出。只接受 'A', 'B', 'C', 'D'。
    """
    # 去除前後空白並轉大寫 (處理 " A", "a ", "  A  " 等情況)
    clean_text = output_text.strip().upper()
    
    # 檢查是否為合法選項
    if clean_text in ['A', 'B', 'C', 'D']:
        return clean_text
    else:
        return None  # 解析失敗

# ==========================================
# 4. 核心函數：提取特徵與標籤 (含過濾功能)
# ==========================================
def extract_features_from_data(data_list, model, tokenizer, desc="Processing"):
    """
    輸入：數據列表 (list of dicts)
    輸出：特徵 Tensor (X), 標籤 Tensor (y)
    邏輯：若解析失敗，則跳過該題 (Drop data)
    """
    features = []
    labels = []
    
    total_count = len(data_list)
    skipped_count = 0
    
    print(f"正在從 {desc} 提取特徵 (共 {total_count} 筆)...")

    for item in tqdm(data_list):
        prompt = format_json_prompt(item)
        ground_truth = item['answer'].strip().upper()

        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

        with torch.no_grad():
            # 1. 提取 Hidden States
            outputs = model(**inputs, output_hidden_states=True)

            # 提取最後 3 層的最後一個 Token，並拼接
            layers_to_cat = []
            for i in [1, 2, 3]:
                layer_vec = outputs.hidden_states[-i][:, -1, :].cpu().to(torch.float32)
                layers_to_cat.append(layer_vec)
            combined_feature = torch.cat(layers_to_cat, dim=1) # [1, 12288]

            # 2. 讓模型生成答案
            generated_ids = model.generate(
                **inputs, 
                max_new_tokens=1, 
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id
            )
            new_token_id = generated_ids[0][-1]
            raw_output = tokenizer.decode(new_token_id)
            
            # === [新增] Parsing 與 過濾邏輯 ===
            pred_text = parse_model_output(raw_output)
            
            if pred_text is None:
                # 解析失敗 (輸出不是 A/B/C/D)，跳過此題
                skipped_count += 1
                continue 
            
            # === 若解析成功，才執行標註 ===
            # 3. 標註 (1=答對/Known, 0=答錯/Unknown)
            label = 1.0 if pred_text == ground_truth else 0.0

            features.append(combined_feature)
            labels.append(label)

    # 顯示過濾報告
    valid_count = len(labels)
    print("\n" + "-"*40)
    print(f"📊 [{desc}] 資料處理報告:")
    print(f"   - 原始總數: {total_count}")
    print(f"   - ❌ 刪除 (Invalid): {skipped_count} 題")
    print(f"   - ✅ 保留 (Valid):   {valid_count} 題")
    print("-" * 40 + "\n")
    
    if valid_count == 0:
        print("⚠️ 警告：沒有任何有效數據！請檢查模型輸出是否正常。")
        return None, None

    # 合併為大 Tensor
    X = torch.cat(features, dim=0)
    y = torch.tensor(labels)
    return X, y

# ==========================================
# 5. 主程式流程
# ==========================================
if __name__ == "__main__":
    print(f"Using device: {DEVICE}")

    # --- A. 讀取 JSON 檔案 ---
    if not os.path.exists(JSON_FILE):
        print(f"錯誤：找不到 {JSON_FILE}！請確認檔案在同一目錄下。")
        sys.exit() # 找不到檔案直接結束
        
    with open(JSON_FILE, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    if 'dev' not in raw_data or 'test' not in raw_data:
        print("錯誤：JSON 檔案中缺少 'dev' 或 'test' 欄位。")
        sys.exit()

    dev_data = raw_data['dev']
    test_data = raw_data['test']

    # --- B. 載入 LLM ---
    print(f"Loading LLM: {MODEL_NAME}...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    model.eval()

    # --- C. 提取特徵 (Data Collection) ---
    # 1. 處理 Dev Set
    X_train, y_train = extract_features_from_data(dev_data, model, tokenizer, desc="Dev Set")

    # 2. 處理 Test Set
    # test_data = test_data[:500] # 若要測試可解開限制
    X_test, y_test = extract_features_from_data(test_data, model, tokenizer, desc="Test Set")

    # 確保有數據才繼續
    if X_train is None or X_test is None:
        print("數據不足，程式終止。")
        sys.exit()

    # --- D. 訓練 Probe (Training) ---
    print("\n" + "="*30)
    print("開始訓練探測器 (Training Probe)...")
    print("="*30)

    # 準備 DataLoader
    train_dataset = TensorDataset(X_train.to(DEVICE), y_train.unsqueeze(1).to(DEVICE))
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    # 初始化 Probe
    input_dim = X_train.shape[1]
    probe = ImprovedProbe(input_dim).to(DEVICE)
    optimizer = optim.Adam(probe.parameters(), lr=LEARNING_RATE)
    criterion = nn.BCELoss()

    # 訓練迴圈
    best_val_acc = 0.0

    for epoch in range(EPOCHS):
        probe.train()
        epoch_loss = 0
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = probe(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        # 每個 Epoch 都在 Test Set 上做一次簡單驗證
        if (epoch + 1) % 5 == 0:
            probe.eval()
            with torch.no_grad():
                test_out = probe(X_test.to(DEVICE))
                test_pred = (test_out > 0.5).float()
                acc = accuracy_score(y_test.cpu(), test_pred.cpu())

                if acc > best_val_acc:
                    best_val_acc = acc

                print(f"Epoch {epoch+1:02d} | Loss: {epoch_loss/len(train_loader):.4f} | Test Acc: {acc:.4f}")

    # --- E. 最終評估 (Evaluation) ---
    print("\n" + "="*30)
    print("最終評估報告 (Final Report)")
    print("="*30)

    probe.eval()
    with torch.no_grad():
        final_probs = probe(X_test.to(DEVICE)).cpu().numpy()
        final_preds = (final_probs > 0.5).astype(int)

    print(classification_report(y_test, final_preds, target_names=["Unknown (0)", "Known (1)"]))
    print(f"Best Test Accuracy during training: {best_val_acc:.4f}")

Using device: cuda
Loading LLM: mistralai/Mistral-7B-v0.1...


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

正在從 Dev Set 提取特徵 (共 1000 筆)...


100%|██████████| 1000/1000 [03:49<00:00,  4.36it/s]



----------------------------------------
📊 [Dev Set] 資料處理報告:
   - 原始總數: 1000
   - ❌ 刪除 (Invalid): 104 題
   - ✅ 保留 (Valid):   896 題
----------------------------------------

正在從 Test Set 提取特徵 (共 1000 筆)...


100%|██████████| 1000/1000 [03:59<00:00,  4.18it/s]



----------------------------------------
📊 [Test Set] 資料處理報告:
   - 原始總數: 1000
   - ❌ 刪除 (Invalid): 84 題
   - ✅ 保留 (Valid):   916 題
----------------------------------------


開始訓練探測器 (Training Probe)...
Epoch 05 | Loss: 0.2789 | Test Acc: 0.6976
Epoch 10 | Loss: 0.1730 | Test Acc: 0.6583
Epoch 15 | Loss: 0.1009 | Test Acc: 0.6878
Epoch 20 | Loss: 0.1167 | Test Acc: 0.7074
Epoch 25 | Loss: 0.0959 | Test Acc: 0.6921
Epoch 30 | Loss: 0.0822 | Test Acc: 0.6758
Epoch 35 | Loss: 0.0487 | Test Acc: 0.6921
Epoch 40 | Loss: 0.1028 | Test Acc: 0.6889
Epoch 45 | Loss: 0.0869 | Test Acc: 0.7009
Epoch 50 | Loss: 0.0309 | Test Acc: 0.7140

最終評估報告 (Final Report)
              precision    recall  f1-score   support

 Unknown (0)       0.57      0.62      0.60       312
   Known (1)       0.80      0.76      0.78       604

    accuracy                           0.71       916
   macro avg       0.68      0.69      0.69       916
weighted avg       0.72      0.71      0.72       916

Best Test Accurac

In [2]:
# ==========================================
# [New Section] 計算論文標準指標 (R-Acc, ER, A-Acc, A-F1)
# ==========================================
print("\n" + "="*40)
print("🏆 論文標準指標評估 (Paper Metrics Calculation)")
print("="*40)

# 1. 準備數據
# y_test 是 tensor, 轉為 numpy array (如果還不是的話)
if isinstance(y_test, torch.Tensor):
    paper_correct_flags = y_test.cpu().numpy().astype(int)
else:
    paper_correct_flags = y_test.astype(int)

# final_probs 是 Probe 認為 "Known (1)" 的機率
# 如果還是 tensor, 轉為 numpy
if isinstance(final_probs, torch.Tensor):
    probs_known = final_probs.cpu().numpy().flatten()
else:
    probs_known = final_probs.flatten()

# 2. 轉換邏輯 (關鍵步驟)
# Probe 預測 1 (Known) -> 系統選擇 "回答" (Abstain Flag = 0)
# Probe 預測 0 (Unknown) -> 系統選擇 "拒絕" (Abstain Flag = 1)

# 這裡使用 0.5 作為閾值，您可以根據需要調整 (例如 0.4 或 0.6)
threshold = 0.5
paper_abstain_flags = (probs_known < threshold).astype(int)

# 拒絕信心分數 (Abstain Score) = 1 - 知道的機率
paper_abstain_scores = 1 - probs_known

# 3. 計算指標函數 (直接內嵌在這裡方便您執行)
def get_paper_metrics(c_flags, a_flags):
    A = 0 # Answered & Correct (True Accept)
    B = 0 # Abstained & Correct (False Abstain - 過度保守)
    C = 0 # Answered & Incorrect (False Accept - 幻覺)
    D = 0 # Abstained & Incorrect (True Abstain - 成功攔截)

    for i in range(len(c_flags)):
        if a_flags[i] == 1: # 拒絕
            if c_flags[i] == 1: B += 1
            else: D += 1
        else: # 回答
            if c_flags[i] == 1: A += 1
            else: C += 1

    total = A + B + C + D

    r_acc = A / (A + C) if (A + C) > 0 else 0.0
    er = (A - C) / total
    a_acc = (A + D) / total

    precision = D / (B + D) if (B + D) > 0 else 0.0
    recall = D / (C + D) if (C + D) > 0 else 0.0
    a_f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return A, B, C, D, r_acc, er, a_acc, a_f1

# 4. 執行計算與顯示
A, B, C, D, r_acc, er, a_acc, a_f1 = get_paper_metrics(paper_correct_flags, paper_abstain_flags)

print(f"總樣本數 (Total Samples): {len(paper_correct_flags)}")
print("-" * 40)
print(f"矩陣細節:")
print(f"  [A] 正確回答 (True Accept):     {A} (目標: 高)")
print(f"  [B] 過度拒絕 (False Abstain):   {B} (目標: 低)")
print(f"  [C] 產生幻覺 (False Accept):    {C} (目標: 低 - 最危險!)")
print(f"  [D] 成功攔截 (True Abstain):    {D} (目標: 高)")
print("-" * 40)
print(f"📊 關鍵指標 (Key Metrics):")
print(f"  1. Reliable Accuracy (R-Acc) : {r_acc:.4f}")
print(f"  2. Effective Reliability (ER): {er:.4f}")
print(f"  3. Abstain Accuracy (A-Acc)  : {a_acc:.4f}")
print(f"  4. Abstain F1 Score (A-F1)   : {a_f1:.4f}")
print("="*40)


🏆 論文標準指標評估 (Paper Metrics Calculation)
總樣本數 (Total Samples): 916
----------------------------------------
矩陣細節:
  [A] 正確回答 (True Accept):     460 (目標: 高)
  [B] 過度拒絕 (False Abstain):   144 (目標: 低)
  [C] 產生幻覺 (False Accept):    118 (目標: 低 - 最危險!)
  [D] 成功攔截 (True Abstain):    194 (目標: 高)
----------------------------------------
📊 關鍵指標 (Key Metrics):
  1. Reliable Accuracy (R-Acc) : 0.7958
  2. Effective Reliability (ER): 0.3734
  3. Abstain Accuracy (A-Acc)  : 0.7140
  4. Abstain F1 Score (A-F1)   : 0.5969
